In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [110]:
class salary_tax_calculator:
    """Tax calculation on salary as per Irish laws"""
    def __init__(self, pay):
        self.netPay = self.pay = pay
        self.tax_credits = 1650*2 # pesonal tax credits exemption
        self.PRSI = self.pay*4/100 # Pay Related Social Insurance (PRSI) @ 4%
        self.tax_bracket = [35300 * 0.8, (self.pay - 35300) * 0.6] # 20% tax on 35,300 euros & 40% tax on remaining income
        self.health_insurance = self.employee_pension_contributions = 0
        
    def set_params(self, health_insurance, employee_pension_contributions = 0, dependant_relative_taxCredit = True):
        self.health_insurance = health_insurance * 12
        self.employee_pension_contributions = employee_pension_contributions/100 * self.pay
        self.pay -= self.employee_pension_contributions
        self.tax_bracket = [35300 * 0.8, (self.pay - 35300) * 0.6] # 20% tax on 35,300 euros & 40% tax on remaining income
        
        if dependant_relative_taxCredit:
            self.tax_credits += 245
        
    def get_employee_pension_contributions(self):
        return self.employee_pension_contributions
        
    def get_USC_rates(self):
        url = 'https://www.citizensinformation.ie/en/money_and_tax/tax/income_tax/universal_social_charge.html'
        df = pd.read_html(url)[0]
        USC_rate = df['Rate'].apply(lambda x: float(x.strip('%'))/100)
        #USC_lim = df['Income band'].str.extract('([\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+)')
        return USC_rate
    
    def get_USC(self):
        USC_rate = self.get_USC_rates() # USC_rate = [0.5, 2, 4.5, 8]
        USC_lim = [12012, 20687, 70044] # USC_lim = int (USC_lim)
        
        if pay > USC_lim[2]:
            USC_tax = (USC_lim[2]-USC_lim[1])*USC_rate[2] + (self.netPay-USC_lim[2])*USC_rate[3]
        else: 
            USC_tax = (self.netPay-USC_lim[1])*USC_rate[2]
        
        return USC_lim[0]*USC_rate[0] + (USC_lim[1]-USC_lim[0])*USC_rate[1] + USC_tax # Universal Social Charge
        
    
    def income_after_tax(self):
        USC = self.get_USC()
        print('=>',self.pay, USC, self.PRSI)

        return sum(self.tax_bracket) + self.tax_credits - self.PRSI - USC

In [111]:
pay = 69000
p1 = salary_tax_calculator(pay)
care_select = 136.60
p1.set_params(care_select, 5, False)
x = p1.income_after_tax()
print(p1.get_employee_pension_contributions())
print('Income = ', x, x/12)

=> 65550.0 2407.645 2760.0
3450.0
Income =  44522.355 3710.1962500000004


In [8]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def get_USC_rates(self):
        url = 'https://www.citizensinformation.ie/en/money_and_tax/tax/income_tax/universal_social_charge.html'
        df = pd.read_html(url)[0]
        USC_rate = df['Rate'].apply(lambda x: float(x.strip('%'))/100)
        #USC_lim = df['Income band'].str.extract('([\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+)')
        
        return USC_rate
    
    def myfunc(self):
        print("Hello my name is " + self.name)
        print(self.get_USC_rates())
        
    def callback(self):
        self.myfunc()

p1 = Person("John", 36)
p1.callback()

Hello my name is John
0    0.005
1    0.020
2    0.045
3    0.080
4    0.110
Name: Rate, dtype: float64
